In [0]:
%pip install databricks-labs-dqx==0.6.0

In [0]:
import json
from pathlib import Path
from functions.utility import apply_job_type, print_settings, schema_exists
from functions.history import build_and_merge_file_history

color = dbutils.widgets.get('color')
job_settings = json.loads(dbutils.widgets.get('job_settings'))
table = job_settings['table']
settings = json.loads(next(Path().glob(f'./layer_*_{color}/{table}.json')).read_text())
settings = apply_job_type(settings)
dst_table_name = settings['dst_table_name']

print_settings(job_settings, settings, color, table)

history = job_settings.get('history', {})
if str(history.get('build_history', 'false')).lower() == 'true':
    full_table_name = history.get('full_table_name', dst_table_name)
    history_schema = history.get('history_schema')
    catalog = full_table_name.split('.')[0]
    if history_schema is None:
        print('Skipping history build: no history_schema provided')
    elif schema_exists(catalog, history_schema, spark):
        build_and_merge_file_history(full_table_name, history_schema, spark)
    else:
        print(f'Skipping history build: schema {catalog}.{history_schema} not found')
